In [1]:
from Analyzer import *

Welcome to JupyROOT 6.16/00


In [2]:
ana = Analyzer()
print(ana.code_event_struct)

struct Events {
    int *HLT_Ele32_WPTight_Gsf;
    int *HLT_IsoMu24;
    int *nElectron;
    int *nMuon;
    float *Electron_pt;
    float *Electron_eta;
    int *Electron_cutBased;
    float *Electron_pfRelIso03_all;
    int *Electron_pdgId;
    float *Muon_pt;
    float *Muon_eta;
    int *Muon_isGlobal;
    int *Muon_isPFcand;
    int *Muon_tightId;
    float *Muon_pfRelIso03_all;
    int *Muon_pdgId;
    int *cumsum_nElectron;
    int *cumsum_nMuon;
    int nev;
};



In [3]:
ana.set_infile('data/tt.root')

In [16]:
ana.clear_infile()

In [8]:
f = open( "Analyzer.cu","r") 
code = f.read()
f.close()

In [10]:
testmod = cu.compiler.SourceModule( ana.code_event_struct + code)

/home/zchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(38): warning: variable "iPassElectron" was set but never used


  """Entry point for launching an IPython kernel.


In [12]:
knl_objectSelection_electrons = testmod.get_function('knl_objectSelection_electrons')

In [59]:
featureConfigs = pd.concat(pd.read_csv(f,index_col="featureName") for f in glob.glob("featureConfig/*.csv"))

In [33]:
# devents = GPUStruct([(np.float32,'*Electron_pt', events['Electron_pt']),
#                      (np.float32,'*Electron_eta',events['Electron_eta']),
#                     ])

# dout = GPUStruct([(np.float32,'*res', np.zeros(578981,dtype=np.float32))
#                     ])
devents.copy_to_gpu()
dout.copy_to_gpu()

In [35]:
knl_objectSelection_electrons = testmod.get_function('knl_objectSelection_electrons')

In [36]:
knl_objectSelection_electrons(devents.get_ptr(), dout.get_ptr(), grid=(int(578981/1024)+1,1,1), block=(1024,1,1))

In [37]:
len(events['Electron_eta'])

578981

In [38]:
devents._GPUStruct__ptr

In [39]:
devents.copy_from_gpu()

In [40]:
0.01*events.Electron_pt+events.Electron_eta

array([ 2.5062249 ,  1.7968841 , -0.02231631, ..., -1.0562634 ,
        0.9958391 , -0.5016483 ], dtype=float32)

In [41]:
dout.res

array([ 2.5062249,  1.7968841, -0.0223163, ..., -1.0562634,  0.9958391,
       -0.5016483], dtype=float32)

In [31]:
dout.copy_from_gpu()

In [21]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-

import numpy as np
import pycuda.driver as cuda
import pycuda.tools as tools
import pycuda.autoinit

from gpustruct import GPUStruct
from mako.template import Template
from pycuda.compiler import SourceModule

src_template = Template(
"""
    struct Dist {
        %for s in xrange(ns):
        float *dist${s};
        %endfor
    };

    // return linear index based on x,y coordinate
    __device__ int get_index(int xcoord, int ycoord)
    {
        return ycoord + xcoord * ${ny};
    };

    __global__ void initialize(float *rho, float *ux, float *uy, Dist *ftmp)
    {
        int idx;
        float dens, velx, vely, vv, ev;

        for (int y = threadIdx.x + blockIdx.x * blockDim.x; 
                 y < ${ny}; 
                 y += blockDim.x * gridDim.x)   
        {   
            for (int x = threadIdx.y + blockIdx.y * blockDim.y; 
                 x < ${nx}; 
                 x += blockDim.y * gridDim.y) 
            {
                if ((x > 0) && (x < ${nx-1}) && (y > 0) && (y < ${ny-1}))
                {
                    idx = get_index(x,y);
                    dens = rho[idx]; velx = ux[idx]; vely = uy[idx];
                    vv = velx*velx + vely*vely;

                    %for s in xrange(ns):
                    // s = ${s}; \vec{e}[${s}] = [${ex[s]},${ey[s]}]
                    float *ftmp${s}_ptr1 = ftmp->dist${s};
                    ev = ${float(ex[s])}f*velx + ${float(ey[s])}f*vely;
                    ftmp${s}_ptr1[idx] = ${w[s]}f*dens*(1.0f+3.0f*ev+4.5f*ev*ev-1.5f*vv);                   
                    %endfor
                }
            }
        }
    }
"""
)

class channelFlow:
    # initialize channelFlow
    def __init__(self, nx, ny):
        self.nx, self.ny = nx, ny

        max_threads_per_block = tools.DeviceData().max_threads
        self.blocksize = (ny if ny<32 else 32, nx if nx<32 else 32, 1)  # threads per block
        self.gridsize = (ny/self.blocksize[0], nx/self.blocksize[1], 1) # blocks per grid

        self.ns = 9
        self.w = np.array([4./9, 1./9, 1./9, 1./9, 1./9, 1./36, 1./36, 1./36, 1./36])
        self.ex = np.array([0, 1, -1, 0, 0, 1, -1, -1, 1])
        self.ey = np.array([0, 0, 0, 1, -1, 1, 1, -1, -1])

        self.ctx = { 'nx': self.nx, 'ny': self.ny, 'ns': self.ns,
                     'w': self.w, 'ex': self.ex, 'ey': self.ey
                     }

        dtype = np.float32
        self.ftmp = np.zeros([self.nx,self.ny,self.ns]).astype(dtype)
        self.rho  = np.zeros([self.nx,self.ny]).astype(dtype)
        self.ux   = np.zeros([self.nx,self.ny]).astype(dtype)
        self.uy   = np.zeros([self.nx,self.ny]).astype(dtype)

        self.ftmp_gpu = GPUStruct([
            (np.float32,'*dist0', self.ftmp[:,:,0]),
            (np.float32,'*dist1', self.ftmp[:,:,1]),
            (np.float32,'*dist2', self.ftmp[:,:,2]),
            (np.float32,'*dist3', self.ftmp[:,:,3]),
            (np.float32,'*dist4', self.ftmp[:,:,4]),
            (np.float32,'*dist5', self.ftmp[:,:,5]),
            (np.float32,'*dist6', self.ftmp[:,:,6]),
            (np.float32,'*dist7', self.ftmp[:,:,7]),
            (np.float32,'*dist8', self.ftmp[:,:,8])
            ])
        self.rho_gpu  = cuda.mem_alloc(self.rho.nbytes)
        self.ux_gpu   = cuda.mem_alloc(self.ux.nbytes)
        self.uy_gpu   = cuda.mem_alloc(self.uy.nbytes)

    def run(self):
        src = src_template.render(**self.ctx)
        code = SourceModule(src)
        initialize  = code.get_function('initialize')

        self.rho[:,:] = 1.
        self.ux[:,:] = 0.
        self.uy[:,:] = 0.

        self.ftmp_gpu.copy_to_gpu()
        cuda.memcpy_htod(self.rho_gpu, self.rho)
        cuda.memcpy_htod(self.ux_gpu, self.ux)
        cuda.memcpy_htod(self.uy_gpu, self.uy)

        initialize(
            self.rho_gpu, self.ux_gpu, self.uy_gpu,
            self.ftmp_gpu.get_ptr(), 
            block=self.blocksize, grid=self.gridsize
            )

        self.dens = np.zeros_like(self.rho)
        cuda.memcpy_dtoh(self.dens, self.rho_gpu)

if __name__ == "__main__":
    sim = channelFlow(64,64); sim.run()

TypeError: 'Undefined' object is not callable